In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import os
import re

# setup paths

In [2]:
!pwd
HOME_dir = os.getcwd(); HOME_dir

/Users/xianli/Desktop/fast


'/Users/xianli/Desktop/fast'

In [3]:
DATA_folder = 'Kaggle/data/rossmann'
data_dir = os.path.join(HOME_dir,DATA_folder)
data_dir

'/Users/xianli/Desktop/fast/Kaggle/data/rossmann'

# Read data

read all the tables together

In [4]:
table_names = ['train', 'store', 'store_states', 'state_names',
               'googletrend', 'weather', 'test']
tables =[pd.read_csv(data_dir+'/'+fnames+'.csv', low_memory=False) for fnames in table_names]
train, store, store_states, state_names, googletrend, weather, test = tables
train.shape, test.shape

((1017209, 9), (41088, 8))

In [5]:
message = ['Shape of the table "%s" is %s' %(table_names[i], str(tb.shape)) 
       for i, tb in enumerate(tables)]
for out in message:
    print(out)

Shape of the table "train" is (1017209, 9)
Shape of the table "store" is (1115, 10)
Shape of the table "store_states" is (1115, 2)
Shape of the table "state_names" is (16, 2)
Shape of the table "googletrend" is (2072, 3)
Shape of the table "weather" is (15840, 24)
Shape of the table "test" is (41088, 8)


In [6]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


### Convert categorical variables to boolean values

In [7]:
train.StateHoliday = train.StateHoliday != '0'
train.SchoolHoliday = train.SchoolHoliday != 0

test.StateHoliday = test.StateHoliday != '0'
test.SchoolHoliday = test.SchoolHoliday != 0

In [8]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,False,True
1,2,5,2015-07-31,6064,625,1,1,False,True
2,3,5,2015-07-31,8314,821,1,1,False,True
3,4,5,2015-07-31,13995,1498,1,1,False,True
4,5,5,2015-07-31,4822,559,1,1,False,True


In [11]:
weather.head().T

,0,1,2,3,4
file,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen
Date,2013-01-01,2013-01-02,2013-01-03,2013-01-04,2013-01-05
Max_TemperatureC,8,7,11,9,8
Mean_TemperatureC,4,4,8,9,8
Min_TemperatureC,2,1,6,8,7
Dew_PointC,7,5,10,9,8
MeanDew_PointC,5,3,8,9,7
Min_DewpointC,1,2,4,8,6
Max_Humidity,94,93,100,100,100
Mean_Humidity,87,85,93,94,94


In [12]:
state_names.head()

,StateName,State
0,BadenWuerttemberg,BW
1,Bayern,BY
2,Berlin,BE
3,Brandenburg,BB
4,Bremen,HB


### Join two dataframes by comparing specified column values

In [13]:
# combining two dataframes based on the values in left_on column 
# in the left dataframe and right_on column in the right df
def join_df(left, right, left_on, right_on=None,suffix='_y'):
    if right_on is None: right_on = left_on
    return left.merge(right,how='left', left_on=left_on,
                         right_on= right_on, suffixes=("",suffix))

In [14]:
weather = join_df(weather, state_names, "file","StateName")

In [15]:
weather.head().T

,0,1,2,3,4
file,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen
Date,2013-01-01,2013-01-02,2013-01-03,2013-01-04,2013-01-05
Max_TemperatureC,8,7,11,9,8
Mean_TemperatureC,4,4,8,9,8
Min_TemperatureC,2,1,6,8,7
Dew_PointC,7,5,10,9,8
MeanDew_PointC,5,3,8,9,7
Min_DewpointC,1,2,4,8,6
Max_Humidity,94,93,100,100,100
Mean_Humidity,87,85,93,94,94


In [16]:
# select rows that contain _y in the StateName column
contain_y = weather.loc[weather.StateName.str.contains('_y',regex=False)]
# check if all the values in a column is equal to False
(weather.StateName.str.contains('_y',regex=False) == False).all()

True

### Add new columns to df by splitting values in other columns

In [17]:
googletrend.head()

,file,week,trend
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67


In [52]:
# googletrend=googletrend.drop(['Date_beg','Date_end'],axis=1)

In [18]:
googletrend['Date'] = googletrend.week.str.split(' - ', expand=True)[0]
googletrend['State'] = googletrend.file.str.split('_',expand=True)[2]
googletrend.head()

,file,week,trend,Date,State
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96,2012-12-02,SN
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95,2012-12-09,SN
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91,2012-12-16,SN
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48,2012-12-23,SN
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67,2012-12-30,SN


In [19]:
# data specific: replace all instances of state name 'NI' to match the usage in the rest of the data: 'HB,NI'.
googletrend.loc[googletrend.State=='NI','State'] = 'HB,NI'

### Split datetime column into several categorical columns to better capture the time (than using strings)
#### Always consider doing this for datetime objects

In [20]:
def add_date_part(df, col_name, drop=True, time=False):
    col = df[col_name]
    col_type = col.dtype
    # check column field data type, good usage
    if isinstance(col_type, pd.core.dtypes.dtypes.DatetimeTZDtype):
        col_type = np.datetime64
    if not np.issubdtype(col_type, np.datetime64): # if col_type is not a subtype of np.datetime64
        df[col_name] = col = pd.to_datetime(col, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$','', col_name) # remove [Dd]ate from col_name
    # these attributes are provided by the pandas datetime object
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: # if time of the date is also available
        attr += ['Hour', 'Minute', 'Second']
    for n in attr: # create new columns
        df[targ_pre + n] = getattr(col.dt, n.lower()) # get atrribute value from pd series object
    df[targ_pre + 'Elapsed'] = col.astype(np.int64) // 10 ** 9
    if drop: df.drop(col_name, axis=1, inplace=True) # drop the original column
        
        

In [21]:
add_date_part(weather,'Date',drop=False)
add_date_part(googletrend, "Date", drop=False)
add_date_part(train, "Date", drop=False)
add_date_part(test, "Date", drop=False)

In [22]:
googletrend.head()

,file,week,trend,Date,State,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96,2012-12-02,SN,2012,12,48,2,6,337,False,False,False,False,False,False,1354406400
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95,2012-12-09,SN,2012,12,49,9,6,344,False,False,False,False,False,False,1355011200
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91,2012-12-16,SN,2012,12,50,16,6,351,False,False,False,False,False,False,1355616000
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48,2012-12-23,SN,2012,12,51,23,6,358,False,False,False,False,False,False,1356220800
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67,2012-12-30,SN,2012,12,52,30,6,365,False,False,False,False,False,False,1356825600


### Extract specific rows from df
The Google trends data has a special category for the whole of the Germany - we'll pull that out so we can use it explicitly.

In [23]:
trend_de = googletrend[googletrend.file=='Rossmann_DE']
trend_de.head(), trend_de.shape

(            file                     week  trend       Date State  Year  \
 148  Rossmann_DE  2012-12-02 - 2012-12-08     77 2012-12-02  None  2012   
 149  Rossmann_DE  2012-12-09 - 2012-12-15     76 2012-12-09  None  2012   
 150  Rossmann_DE  2012-12-16 - 2012-12-22     85 2012-12-16  None  2012   
 151  Rossmann_DE  2012-12-23 - 2012-12-29     59 2012-12-23  None  2012   
 152  Rossmann_DE  2012-12-30 - 2013-01-05     61 2012-12-30  None  2012   
 
      Month  Week  Day  Dayofweek  Dayofyear  Is_month_end  Is_month_start  \
 148     12    48    2          6        337         False           False   
 149     12    49    9          6        344         False           False   
 150     12    50   16          6        351         False           False   
 151     12    51   23          6        358         False           False   
 152     12    52   30          6        365         False           False   
 
      Is_quarter_end  Is_quarter_start  Is_year_end  Is_year_start     E

## Outer join all dataframes
- note difference between inner, left, right and full outer joins
- In left join, use values from the left table as keys. If any key is not found in the right table, the corresponding right table fields will be set to null in the new table --> can used to check whether data is complete
- it's key --> value lookups

In [24]:
store = join_df(store, store_states, "Store") # left join
len(store[store.State.isnull()]) # if any missing keys in the right table

0

In [25]:
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,State
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,HE
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",TH
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",NW
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN,BE
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN,SN


In [26]:
# merging dataframes based on the same stores
joined = join_df(train,store,"Store")
joined_test = join_df(test,store,"Store")
len(joined[joined.StoreType.isnull()]),len(joined_test[joined_test.StoreType.isnull()])

(0, 0)

In [27]:
joined.head().T

,0,1,2,3,4
Store,1,2,3,4,5
DayOfWeek,5,5,5,5,5
Date,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00
Sales,5263,6064,8314,13995,4822
Customers,555,625,821,1498,559
Open,1,1,1,1,1
Promo,1,1,1,1,1
StateHoliday,False,False,False,False,False
SchoolHoliday,True,True,True,True,True
Year,2015,2015,2015,2015,2015


In [28]:
# join field when all 3 keys are satisfied simultaneously?
joined = join_df(joined, googletrend, ['State', 'Year', 'Week'])
joined_test = join_df(joined_test, googletrend, ["State","Year", "Week"])
len(joined[joined.trend.isnull()]),len(joined_test[joined_test.trend.isnull()])

(0, 0)

In [29]:
joined.head().T

,0,1,2,3,4
Store,1,2,3,4,5
DayOfWeek,5,5,5,5,5
Date,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00
Sales,5263,6064,8314,13995,4822
Customers,555,625,821,1498,559
Open,1,1,1,1,1
Promo,1,1,1,1,1
StateHoliday,False,False,False,False,False
SchoolHoliday,True,True,True,True,True
Year,2015,2015,2015,2015,2015


In [30]:
joined.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'Year', 'Month', 'Week', 'Day',
       'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start',
       'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start',
       'Elapsed', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'State', 'file',
       'week', 'trend', 'Date_y', 'Month_y', 'Day_y', 'Dayofweek_y',
       'Dayofyear_y', 'Is_month_end_y', 'Is_month_start_y', 'Is_quarter_end_y',
       'Is_quarter_start_y', 'Is_year_end_y', 'Is_year_start_y', 'Elapsed_y'],
      dtype='object')

In [31]:
joined = joined.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))
joined_test = joined_test.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))
len(joined[joined.trend_DE.isnull()]),len(joined_test[joined_test.trend_DE.isnull()])

(0, 0)

In [32]:
joined = join_df(joined, weather, ["State","Date"])
joined_test = join_df(joined_test, weather, ["State","Date"])
len(joined[joined.Mean_TemperatureC.isnull()]),len(joined_test[joined_test.Mean_TemperatureC.isnull()])

(0, 0)

In [33]:
joined.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,...,Day_y,Dayofweek_y,Dayofyear_y,Is_month_end_y,Is_month_start_y,Is_quarter_end_y,Is_quarter_start_y,Is_year_end_y,Is_year_start_y,Elapsed_y
0,1,5,2015-07-31,5263,555,1,1,False,True,2015,...,31,4,212,True,False,False,False,False,False,1438300800
1,2,5,2015-07-31,6064,625,1,1,False,True,2015,...,31,4,212,True,False,False,False,False,False,1438300800
2,3,5,2015-07-31,8314,821,1,1,False,True,2015,...,31,4,212,True,False,False,False,False,False,1438300800
3,4,5,2015-07-31,13995,1498,1,1,False,True,2015,...,31,4,212,True,False,False,False,False,False,1438300800
4,5,5,2015-07-31,4822,559,1,1,False,True,2015,...,31,4,212,True,False,False,False,False,False,1438300800


In [34]:
joined.shape, joined.columns

((1017209, 100),
 Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
        'StateHoliday', 'SchoolHoliday', 'Year', 'Month', 'Week', 'Day',
        'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start',
        'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start',
        'Elapsed', 'StoreType', 'Assortment', 'CompetitionDistance',
        'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
        'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'State', 'file',
        'week', 'trend', 'Date_y', 'Month_y', 'Day_y', 'Dayofweek_y',
        'Dayofyear_y', 'Is_month_end_y', 'Is_month_start_y', 'Is_quarter_end_y',
        'Is_quarter_start_y', 'Is_year_end_y', 'Is_year_start_y', 'Elapsed_y',
        'file_DE', 'week_DE', 'trend_DE', 'Date_DE', 'State_DE', 'Month_DE',
        'Day_DE', 'Dayofweek_DE', 'Dayofyear_DE', 'Is_month_end_DE',
        'Is_month_start_DE', 'Is_quarter_end_DE', 'Is_quarter_start_DE',
        'Is_ye

#### Drop certain columns

In [35]:
for df in (joined, joined_test):
    for c in df.columns:
        if c.endswith('_y'):
            if c in df.columns:
                df.drop(c,inplace=True,axis = 1)

In [36]:
joined.shape, joined.columns

((1017209, 74),
 Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
        'StateHoliday', 'SchoolHoliday', 'Year', 'Month', 'Week', 'Day',
        'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start',
        'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start',
        'Elapsed', 'StoreType', 'Assortment', 'CompetitionDistance',
        'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
        'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'State', 'file',
        'week', 'trend', 'file_DE', 'week_DE', 'trend_DE', 'Date_DE',
        'State_DE', 'Month_DE', 'Day_DE', 'Dayofweek_DE', 'Dayofyear_DE',
        'Is_month_end_DE', 'Is_month_start_DE', 'Is_quarter_end_DE',
        'Is_quarter_start_DE', 'Is_year_end_DE', 'Is_year_start_DE',
        'Elapsed_DE', 'Max_TemperatureC', 'Mean_TemperatureC',
        'Min_TemperatureC', 'Dew_PointC', 'MeanDew_PointC', 'Min_DewpointC',
        'Max_Humidity', 'Mean_Humidity', '

In [37]:
df.head().T

,0,1,2,3,4
Id,1,2,3,4,5
Store,1,3,7,8,9
DayOfWeek,4,4,4,4,4
Date,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00
Open,1,1,1,1,1
Promo,1,1,1,1,1
StateHoliday,False,False,False,False,False
SchoolHoliday,False,False,False,False,False
Year,2015,2015,2015,2015,2015
Month,9,9,9,9,9


# Fill missing values

- Fill NAs in some columns with a value that doesn't otherwise appear in the data

In [38]:
print(np.sort(df['CompetitionOpenSinceYear'].unique()))
print(np.sort(df['CompetitionOpenSinceMonth'].unique()))
print(np.sort(df['Promo2SinceYear'].unique()))
print(np.sort(df['Promo2SinceWeek'].unique()))

[1900. 1961. 1990. 1994. 1995. 1998. 1999. 2000. 2001. 2002. 2003. 2004.
 2005. 2006. 2007. 2008. 2009. 2010. 2011. 2012. 2013. 2014. 2015.   nan]
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. nan]
[2009. 2010. 2011. 2012. 2013. 2014. 2015.   nan]
[ 1.  5.  9. 10. 13. 14. 18. 22. 23. 27. 28. 31. 35. 36. 37. 39. 40. 44.
 45. 48. 49. nan]


In [39]:
for df in (joined, joined_test):
    df['CompetitionOpenSinceYear'] = df.CompetitionOpenSinceYear.fillna(1900).astype(np.int32)
    df['CompetitionOpenSinceMonth'] = df.CompetitionOpenSinceMonth.fillna(1).astype(np.int32)
    df['Promo2SinceYear'] = df.Promo2SinceYear.fillna(1900).astype(np.int32)
    df['Promo2SinceWeek'] = df.Promo2SinceWeek.fillna(1).astype(np.int32)

### Combining info from columns to form "CompetitionOpenSince" and "CompetitionDaysOpen" columns

In [40]:
for df in (joined,joined_test):
    
    # make this new datetime column
    df["CompetitionOpenSince"] = pd.to_datetime(dict(year=df.CompetitionOpenSinceYear,
                                               month=df.CompetitionOpenSinceMonth,
                                               day=15))
    
    # calculate days since competiton open, note df.Date is the Date column
    df["CompetitionDaysOpen"] = df.Date.subtract(df.CompetitionOpenSince).dt.days


In [41]:
joined.head().T

,0,1,2,3,4
Store,1,2,3,4,5
DayOfWeek,5,5,5,5,5
Date,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00
Sales,5263,6064,8314,13995,4822
Customers,555,625,821,1498,559
Open,1,1,1,1,1
Promo,1,1,1,1,1
StateHoliday,False,False,False,False,False
SchoolHoliday,True,True,True,True,True
Year,2015,2015,2015,2015,2015


### Replace erroneous data (lookup rows)

In [42]:
for df in (joined,joined_test):
    df.loc[df.CompetitionDaysOpen<0, "CompetitionDaysOpen"] = 0
    df.loc[df.CompetitionOpenSinceYear < 1990, "CompetitionDaysOpen"] = 0


### Convert days to months, and cap the the maximnum time to 2 years to limit the number of unique categories

In [43]:
for df in (joined, joined_test):
    df['CompetitionMonthsOpen'] = df['CompetitionDaysOpen']//30
    df.loc[df.CompetitionMonthsOpen>24, 'CompetitionMonthsOpen'] = 24

joined.CompetitionMonthsOpen.unique()



array([24,  3, 19,  9,  0, 16, 17,  7, 15, 22, 11, 13,  2, 23, 12,  4, 10,
        1, 14, 20,  8, 18,  6, 21,  5])

### Use isoweek module to process Promo dates

In [41]:
# import sys
# !{sys.executable} -m pip install isoweek

In [44]:
from isoweek import Week
for df in (joined,joined_test):
    df["Promo2Since"] = pd.to_datetime(df.apply(lambda x: Week(
    x.Promo2SinceYear, x.Promo2SinceWeek).monday(), axis=1))
    df["Promo2Days"] = df.Date.subtract(df['Promo2Since']).dt.days

In [45]:
for df in (joined,joined_test):
    df.loc[df.Promo2Days<0, "Promo2Days"] = 0
    df.loc[df.Promo2SinceYear<1990, "Promo2Days"] = 0
    df["Promo2Weeks"] = df["Promo2Days"]//7
    df.loc[df.Promo2Weeks<0, "Promo2Weeks"] = 0
    df.loc[df.Promo2Weeks>25, "Promo2Weeks"] = 25
    df.Promo2Weeks.unique()

In [46]:
joined.head().T

,0,1,2,3,4
Store,1,2,3,4,5
DayOfWeek,5,5,5,5,5
Date,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00
Sales,5263,6064,8314,13995,4822
Customers,555,625,821,1498,559
Open,1,1,1,1,1
Promo,1,1,1,1,1
StateHoliday,False,False,False,False,False
SchoolHoliday,True,True,True,True,True
Year,2015,2015,2015,2015,2015


In [45]:
joined.to_pickle(data_dir+'/joined')
joined_test.to_pickle(data_dir+'/joined_test')

# Calculate durations

In [47]:
def get_elapsed(col, pre):
    day1 = np.timedelta64(1,'D')
    last_date = np.datetime64()
    last_store = 0
    res = []
    
    for s,v,d in zip(df.Store.values, df[col].values, df.Date.values):
        if s!=last_store:
            last_date = np.datetime64()
            last_store =s
        if v: last_date = d
        res.append(((d-last_date).astype('timedelta64[D]')/day1))
    df[pre+col] = res

### try on a data subset

In [48]:
columns = ["Date", "Store", "Promo",
           "StateHoliday", "SchoolHoliday"]

In [49]:
df = train[columns].append(test[columns])


In [50]:
df.shape

(1058297, 5)

In [51]:
df.head(20)

,Date,Store,Promo,StateHoliday,SchoolHoliday
0,2015-07-31,1,1,False,True
1,2015-07-31,2,1,False,True
2,2015-07-31,3,1,False,True
3,2015-07-31,4,1,False,True
4,2015-07-31,5,1,False,True
5,2015-07-31,6,1,False,True
6,2015-07-31,7,1,False,True
7,2015-07-31,8,1,False,True
8,2015-07-31,9,1,False,True
9,2015-07-31,10,1,False,True


In [52]:
# sort values by Store first and then by Date
fld = 'SchoolHoliday'
df = df.sort_values(['Store','Date'])
df.head(20)

,Date,Store,Promo,StateHoliday,SchoolHoliday
1016095,2013-01-01,1,0,True,True
1014980,2013-01-02,1,0,False,True
1013865,2013-01-03,1,0,False,True
1012750,2013-01-04,1,0,False,True
1011635,2013-01-05,1,0,False,True
1010520,2013-01-06,1,0,False,True
1009405,2013-01-07,1,1,False,True
1008290,2013-01-08,1,1,False,True
1007175,2013-01-09,1,1,False,True
1006060,2013-01-10,1,1,False,True


In [53]:
get_elapsed(fld,'After')
df.head(20)

,Date,Store,Promo,StateHoliday,SchoolHoliday,AfterSchoolHoliday
1016095,2013-01-01,1,0,True,True,0.0
1014980,2013-01-02,1,0,False,True,0.0
1013865,2013-01-03,1,0,False,True,0.0
1012750,2013-01-04,1,0,False,True,0.0
1011635,2013-01-05,1,0,False,True,0.0
1010520,2013-01-06,1,0,False,True,0.0
1009405,2013-01-07,1,1,False,True,0.0
1008290,2013-01-08,1,1,False,True,0.0
1007175,2013-01-09,1,1,False,True,0.0
1006060,2013-01-10,1,1,False,True,0.0


In [54]:
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')
df.head(20)

,Date,Store,Promo,StateHoliday,SchoolHoliday,AfterSchoolHoliday,BeforeSchoolHoliday
0,2015-09-17,1,1,False,False,13.0,NaN
856,2015-09-16,1,1,False,False,12.0,NaN
1712,2015-09-15,1,1,False,False,11.0,NaN
2568,2015-09-14,1,1,False,False,10.0,NaN
3424,2015-09-13,1,0,False,False,9.0,NaN
4280,2015-09-12,1,0,False,False,8.0,NaN
5136,2015-09-11,1,0,False,False,7.0,NaN
5992,2015-09-10,1,0,False,False,6.0,NaN
6848,2015-09-09,1,0,False,False,5.0,NaN
7704,2015-09-08,1,0,False,False,4.0,NaN


In [55]:
fld = 'StateHoliday'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')
df.head(20)

,Date,Store,Promo,StateHoliday,SchoolHoliday,AfterSchoolHoliday,BeforeSchoolHoliday,AfterStateHoliday,BeforeStateHoliday
0,2015-09-17,1,1,False,False,13.0,NaN,105.0,NaN
856,2015-09-16,1,1,False,False,12.0,NaN,104.0,NaN
1712,2015-09-15,1,1,False,False,11.0,NaN,103.0,NaN
2568,2015-09-14,1,1,False,False,10.0,NaN,102.0,NaN
3424,2015-09-13,1,0,False,False,9.0,NaN,101.0,NaN
4280,2015-09-12,1,0,False,False,8.0,NaN,100.0,NaN
5136,2015-09-11,1,0,False,False,7.0,NaN,99.0,NaN
5992,2015-09-10,1,0,False,False,6.0,NaN,98.0,NaN
6848,2015-09-09,1,0,False,False,5.0,NaN,97.0,NaN
7704,2015-09-08,1,0,False,False,4.0,NaN,96.0,NaN


In [56]:
fld = 'Promo'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')
df.head(20)

,Date,Store,Promo,StateHoliday,SchoolHoliday,AfterSchoolHoliday,BeforeSchoolHoliday,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo
0,2015-09-17,1,1,False,False,13.0,NaN,105.0,NaN,0.0,0.0
856,2015-09-16,1,1,False,False,12.0,NaN,104.0,NaN,0.0,0.0
1712,2015-09-15,1,1,False,False,11.0,NaN,103.0,NaN,0.0,0.0
2568,2015-09-14,1,1,False,False,10.0,NaN,102.0,NaN,0.0,0.0
3424,2015-09-13,1,0,False,False,9.0,NaN,101.0,NaN,9.0,-1.0
4280,2015-09-12,1,0,False,False,8.0,NaN,100.0,NaN,8.0,-2.0
5136,2015-09-11,1,0,False,False,7.0,NaN,99.0,NaN,7.0,-3.0
5992,2015-09-10,1,0,False,False,6.0,NaN,98.0,NaN,6.0,-4.0
6848,2015-09-09,1,0,False,False,5.0,NaN,97.0,NaN,5.0,-5.0
7704,2015-09-08,1,0,False,False,4.0,NaN,96.0,NaN,4.0,-6.0


In [57]:
df = df.set_index('Date')

In [58]:
# set NAs to 0
columns = ['SchoolHoliday', 'StateHoliday', 'Promo']
for o in ['Before', 'After']:
    for p in columns:
        a = o+p
        df[a] = df[a].fillna(0).astype(int)

In [59]:
df.head(20)

,Store,Promo,StateHoliday,SchoolHoliday,AfterSchoolHoliday,BeforeSchoolHoliday,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo
Date,,,,,,,,,,
2015-09-17,1,1,False,False,13,0,105,0,0,0
2015-09-16,1,1,False,False,12,0,104,0,0,0
2015-09-15,1,1,False,False,11,0,103,0,0,0
2015-09-14,1,1,False,False,10,0,102,0,0,0
2015-09-13,1,0,False,False,9,0,101,0,9,-1
2015-09-12,1,0,False,False,8,0,100,0,8,-2
2015-09-11,1,0,False,False,7,0,99,0,7,-3
2015-09-10,1,0,False,False,6,0,98,0,6,-4
2015-09-09,1,0,False,False,5,0,97,0,5,-5


## Calculate rolling values

In [60]:
bwd = df[['Store']+columns].sort_index().groupby("Store").rolling(7, min_periods=1).sum()

In [61]:
fwd = df[['Store']+columns].sort_index(ascending=False
                                      ).groupby("Store").rolling(7, min_periods=1).sum()


In [62]:
bwd

Store  SchoolHoliday  StateHoliday  Promo
Store Date                                                  
1     2013-01-01     1.0            1.0           1.0    0.0
      2013-01-02     2.0            2.0           1.0    0.0
      2013-01-03     3.0            3.0           1.0    0.0
      2013-01-04     4.0            4.0           1.0    0.0
      2013-01-05     5.0            5.0           1.0    0.0
      2013-01-06     6.0            6.0           1.0    0.0
      2013-01-07     7.0            7.0           1.0    1.0
      2013-01-08     7.0            7.0           0.0    2.0
      2013-01-09     7.0            7.0           0.0    3.0
      2013-01-10     7.0            7.0           0.0    4.0
      2013-01-11     7.0            7.0           0.0    5.0
      2013-01-12     7.0            6.0           0.0    5.0
      2013-01-13     7.0            5.0           0.0    5.0
      2013-01-14     7.0            4.0           0.0    4.0
      2013-01-15     7.0            3.0           0.0    3.0
      2013-01-16     7.0            2.0           0.0    2.0
      2013-01-17     7.0            1.0           0.0    1.0
      2013-01-18     7.0            0.0           0.0    0.0
      2013-01-19     7.0            0.0           0.0    0.0
      2013-01-20     7.0            0.0           0.0    0.0
      2013-01-21     7.0            0.0           0.0    1.0
      2013-01-22     7.0            0.0           0.0    2.0
      2013-01-23     7.0            0.0           0.0    3.0
      2013-01-24     7.0            0.0           0.0    4.0
      2013-01-25     7.0            0.0           0.0    5.0
      2013-01-26     7.0            0.0           0.0    5.0
      2013-01-27     7.0            0.0           0.0    5.0
      2013-01-28     7.0            0.0           0.0    4.0
      2013-01-29     7.0            0.0           0.0    3.0
      2013-01-30     7.0            0.0           0.0    2.0
...                  ...            ...           ...    ...
1115  2015-08-19  7805.0            7.0           0.0    3.0
      2015-08-20  7805.0            7.0           0.0    4.0
      2015-08-21  7805.0            7.0           0.0    5.0
      2015-08-22  7805.0            7.0           0.0    5.0
      2015-08-23  7805.0            7.0           0.0    5.0
      2015-08-24  7805.0            7.0           0.0    4.0
      2015-08-25  7805.0            7.0           0.0    3.0
      2015-08-26  7805.0            7.0           0.0    2.0
      2015-08-27  7805.0            7.0           0.0    1.0
      2015-08-28  7805.0            7.0           0.0    0.0
      2015-08-29  7805.0            7.0           0.0    0.0
      2015-08-30  7805.0            7.0           0.0    0.0
      2015-08-31  7805.0            7.0           0.0    1.0
      2015-09-01  7805.0            7.0           0.0    2.0
      2015-09-02  7805.0            7.0           0.0    3.0
      2015-09-03  7805.0            7.0           0.0    4.0
      2015-09-04  7805.0            7.0           0.0    5.0
      2015-09-05  7805.0            6.0           0.0    5.0
      2015-09-06  7805.0            5.0           0.0    5.0
      2015-09-07  7805.0            4.0           0.0    4.0
      2015-09-08  7805.0            3.0           0.0    3.0
      2015-09-09  7805.0            2.0           0.0    2.0
      2015-09-10  7805.0            1.0           0.0    1.0
      2015-09-11  7805.0            0.0           0.0    0.0
      2015-09-12  7805.0            0.0           0.0    0.0
      2015-09-13  7805.0            0.0           0.0    0.0
      2015-09-14  7805.0            0.0           0.0    1.0
      2015-09-15  7805.0            0.0           0.0    2.0
      2015-09-16  7805.0            0.0           0.0    3.0
      2015-09-17  7805.0            0.0           0.0    4.0

[1058297 rows x 4 columns]

In [63]:
bwd.drop('Store',1,inplace=True)
bwd.reset_index(inplace=True)

In [64]:
fwd.drop('Store',1,inplace=True)
fwd.reset_index(inplace=True)

In [65]:
df.reset_index(inplace=True)


In [66]:
bwd

,Store,Date,SchoolHoliday,StateHoliday,Promo
0,1,2013-01-01,1.0,1.0,0.0
1,1,2013-01-02,2.0,1.0,0.0
2,1,2013-01-03,3.0,1.0,0.0
3,1,2013-01-04,4.0,1.0,0.0
4,1,2013-01-05,5.0,1.0,0.0
5,1,2013-01-06,6.0,1.0,0.0
6,1,2013-01-07,7.0,1.0,1.0
7,1,2013-01-08,7.0,0.0,2.0
8,1,2013-01-09,7.0,0.0,3.0
9,1,2013-01-10,7.0,0.0,4.0


In [67]:
df = df.merge(bwd, 'left', ['Date', 'Store'], suffixes=['', '_bw'])
df = df.merge(fwd, 'left', ['Date', 'Store'], suffixes=['', '_fw'])

In [68]:
df.drop(columns,1,inplace=True)


In [69]:
df.head()


,Date,Store,AfterSchoolHoliday,BeforeSchoolHoliday,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo,SchoolHoliday_bw,StateHoliday_bw,Promo_bw,SchoolHoliday_fw,StateHoliday_fw,Promo_fw
0,2015-09-17,1,13,0,105,0,0,0,0.0,0.0,4.0,0.0,0.0,1.0
1,2015-09-16,1,12,0,104,0,0,0,0.0,0.0,3.0,0.0,0.0,2.0
2,2015-09-15,1,11,0,103,0,0,0,0.0,0.0,2.0,0.0,0.0,3.0
3,2015-09-14,1,10,0,102,0,0,0,0.0,0.0,1.0,0.0,0.0,4.0
4,2015-09-13,1,9,0,101,0,9,-1,0.0,0.0,0.0,0.0,0.0,4.0


In [71]:
df.to_pickle(data_dir+'/df')
df["Date"] = pd.to_datetime(df.Date)

In [70]:
joined = pd.read_pickle(data_dir+'/joined')
joined_test = pd.read_pickle(data_dir+'/joined_test')

In [71]:
joined = join_df(joined, df, ['Store', 'Date'])
joined_test = join_df(joined_test, df, ['Store', 'Date'])

In [72]:
joined = joined[joined.Sales!=0]

In [73]:
joined.head().T

,0,1,2,3,4
Store,1,2,3,4,5
DayOfWeek,5,5,5,5,5
Date,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00
Sales,5263,6064,8314,13995,4822
Customers,555,625,821,1498,559
Open,1,1,1,1,1
Promo,1,1,1,1,1
StateHoliday,False,False,False,False,False
SchoolHoliday,True,True,True,True,True
Year,2015,2015,2015,2015,2015


In [74]:
joined.reset_index(inplace=True)
joined_test.reset_index(inplace=True)

In [77]:
joined.to_pickle(data_dir+'/train_clean')
joined_test.to_pickle(data_dir+'/test_clean')

# Get a sample to experiment with

In [80]:
train_df = joined.copy()
del joined

In [81]:
n = len(train_df); n

844338

In [82]:
# generate index of a random sample of size 2000
idx = np.random.permutation(range(n))[:2000] 
idx.sort()

In [83]:
# both loc and iloc slice rows if a single argument is given
small_train_df = train_df.iloc[idx[:1000]]
small_test_df = train_df.iloc[idx[1000:]]

In [85]:
small_train_df.head().T

,376,1543,2228,2894,2988
index,377,1546,2232,2899,2993
Store,378,432,3,670,764
DayOfWeek,5,4,3,3,3
Date,2015-07-31 00:00:00,2015-07-30 00:00:00,2015-07-29 00:00:00,2015-07-29 00:00:00,2015-07-29 00:00:00
Sales,8828,13839,7610,7557,4594
Customers,1267,1726,716,716,453
Open,1,1,1,1,1
Promo,1,1,1,1,1
StateHoliday,False,False,False,False,False
SchoolHoliday,True,True,True,False,True


### select subset of variables as samples

In [86]:
small_cont_vars = ['CompetitionDistance', 'Mean_Humidity'] # continuous
small_cat_vars =  ['Store', 'DayOfWeek', 'PromoInterval'] # categorical

small_train_df = small_train_df[small_cat_vars + small_cont_vars + ['Sales']]
small_test_df = small_test_df[small_cat_vars + small_cont_vars + ['Sales']]


In [87]:
small_train_df.head()

,Store,DayOfWeek,PromoInterval,CompetitionDistance,Mean_Humidity,Sales
376,378,5,NaN,2140.0,61,8828
1543,432,4,NaN,810.0,81,13839
2228,3,3,"Jan,Apr,Jul,Oct",14130.0,67,7610
2894,670,3,"Feb,May,Aug,Nov",2060.0,77,7557
2988,764,3,"Mar,Jun,Sept,Dec",26490.0,63,4594


In [88]:
small_test_df.head()

,Store,DayOfWeek,PromoInterval,CompetitionDistance,Mean_Humidity,Sales
429486,886,1,NaN,12430.0,79,6410
429555,955,1,"Mar,Jun,Sept,Dec",1690.0,79,4590
429637,1037,1,NaN,150.0,79,6984
429763,19,6,"Mar,Jun,Sept,Dec",3240.0,77,4416
429931,188,6,"Feb,May,Aug,Nov",850.0,83,4550
